## feature eng

### 1차 가공

In [484]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import urllib.request
import urllib.request
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tqdm import tqdm
import os
from imblearn.over_sampling import SMOTE

print('-------------------')
print('|     lab1         |')
print('-------------------')

if os.path.exists('/semi.csv'):
    '''
    혹시 파일이 생기지 않는다면, 아래 두 줄의 스크립스틑 파이썬 .py파일로 만들어서 실행하면 됩니다. 
    '''
    url = "https://drive.google.com/uc?export=download&id=1XCU0eo2xZ03xhxJhdrCnVjduCoaBQ7kJ"
    urllib.request.urlretrieve(url, "semi.csv")  # save in a file
else:
    print('data already exist')


-------------------
|     lab1         |
-------------------
data already exist


In [485]:
df = pd.read_csv('semi.csv')

In [486]:
df.isnull().sum().sum()


41951

In [487]:
df.isna().sum().sum()

41951

In [488]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1567 entries, 0 to 1566
Columns: 592 entries, Time to Pass/Fail
dtypes: float64(590), int64(1), object(1)
memory usage: 7.1+ MB


In [489]:
df.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,Pass/Fail
count,1561.000000,1560.000000,1553.000000,1553.000000,1553.000000,1553.0,1553.000000,1558.000000,1565.000000,1565.000000,1565.000000,1565.000000,1565.000000,1564.0,1564.000000,1564.000000,1564.000000,1564.000000,1564.000000,1557.000000,1567.000000,1565.000000,1565.000000,1565.000000,1565.000000,1565.000000,1565.000000,1565.000000,1565.000000,1565.000000,1565.000000,1565.000000,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000,1543.000000,1543.000000,1566.0,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000,1566.0,1566.000000,1566.000000,1566.0,1563.000000,1563.000000,1563.000000,1563.000000,1563.000000,1563.000000,1560.000000,1561.000000,1561.000000,1561.000000,1560.000000,1560.000000,1560.000000,1561.000000,1561.000000,1561.000000,1561.0,1561.000000,1561.000000,773.000000,773.000000,1561.000000,1543.000000,1543.000000,1543.000000,1543.000000,1543.000000,1543.000000,1543.000000,1543.000000,1566.000000,1555.000000,226.000000,1567.000000,1567.000000,1567.000000,1516.000000,1516.000000,1561.000000,1565.000000,1565.000000,1561.000000,1561.000000,1561.000000,1561.0,1561.000000,1561.00000,1561.000000,1561.000000,1561.000000,1565.000000,1565.000000,1561.000000,1561.000000,1561.000000,1561.000000,549.000000,549.000000,549.000000,852.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1543.000000,1567.000000,1567.000000,1558.000000,1558.00000,1558.000000,1558.000000,1558.000000,1558.000000,1558.000000,1558.000000,1558.000000,1558.000000,1558.000000,1559.000000,1559.000000,1559.000000,1562.000000,1561.000000,1560.000000,1553.000000,1553.000000,1553.000000,1553.0,1553.000000,1558.000000,1565.000000,1565.000000,1565.000000,1565.000000,1565.0

In [491]:
df.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,Pass/Fail
count,1561.000000,1560.000000,1553.000000,1553.000000,1553.000000,1553.0,1553.000000,1558.000000,1565.000000,1565.000000,1565.000000,1565.000000,1565.000000,1564.0,1564.000000,1564.000000,1564.000000,1564.000000,1564.000000,1557.000000,1567.000000,1565.000000,1565.000000,1565.000000,1565.000000,1565.000000,1565.000000,1565.000000,1565.000000,1565.000000,1565.000000,1565.000000,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000,1543.000000,1543.000000,1566.0,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000,1566.0,1566.000000,1566.000000,1566.0,1563.000000,1563.000000,1563.000000,1563.000000,1563.000000,1563.000000,1560.000000,1561.000000,1561.000000,1561.000000,1560.000000,1560.000000,1560.000000,1561.000000,1561.000000,1561.000000,1561.0,1561.000000,1561.000000,773.000000,773.000000,1561.000000,1543.000000,1543.000000,1543.000000,1543.000000,1543.000000,1543.000000,1543.000000,1543.000000,1566.000000,1555.000000,226.000000,1567.000000,1567.000000,1567.000000,1516.000000,1516.000000,1561.000000,1565.000000,1565.000000,1561.000000,1561.000000,1561.000000,1561.0,1561.000000,1561.00000,1561.000000,1561.000000,1561.000000,1565.000000,1565.000000,1561.000000,1561.000000,1561.000000,1561.000000,549.000000,549.000000,549.000000,852.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1543.000000,1567.000000,1567.000000,1558.000000,1558.00000,1558.000000,1558.000000,1558.000000,1558.000000,1558.000000,1558.000000,1558.000000,1558.000000,1558.000000,1559.000000,1559.000000,1559.000000,1562.000000,1561.000000,1560.000000,1553.000000,1553.000000,1553.000000,1553.0,1553.000000,1558.000000,1565.000000,1565.000000,1565.000000,1565.000000,1565.0

In [492]:
df.isna().sum().sum()

41951

In [493]:
def DuplicatedHandler(df: pd.DataFrame):
    features = range(590)
    stack = []
    idx = []
    for i, fe in enumerate(features):
        st = df[str(fe)].std()
        me = df[str(fe)].mean()
        mx = df[str(fe)].max()
        mn = df[str(fe)].mean()
        if (st, me, mx, mn) in stack:
            idx.append(fe)
            print('duplicated!', i)
        else:
            stack.append((st, me, mx, mn))

    for i in idx:
        df = df.drop(str(i), axis=1)
    df = df.drop(['Time'], axis=1)
    return df


df = DuplicatedHandler(df)
feature = df.drop('Pass/Fail', axis=1)
target = df['Pass/Fail']
X_train, X_test, y_train, y_test = train_test_split(feature, target, \
                                                    test_size=0.2, random_state=11, stratify=target)

X_train, X_test, y_train, y_test = pd.DataFrame(X_train), pd.DataFrame(X_test), y_train, y_test

duplicated! 52
duplicated! 69
duplicated! 97
duplicated! 141
duplicated! 149
duplicated! 178
duplicated! 179
duplicated! 186
duplicated! 189
duplicated! 190
duplicated! 191
duplicated! 192
duplicated! 193
duplicated! 194
duplicated! 226
duplicated! 229
duplicated! 230
duplicated! 231
duplicated! 232
duplicated! 233
duplicated! 234
duplicated! 235
duplicated! 236
duplicated! 237
duplicated! 240
duplicated! 241
duplicated! 242
duplicated! 243
duplicated! 256
duplicated! 257
duplicated! 258
duplicated! 259
duplicated! 260
duplicated! 261
duplicated! 262
duplicated! 263
duplicated! 264
duplicated! 265
duplicated! 266
duplicated! 276
duplicated! 284
duplicated! 313
duplicated! 314
duplicated! 315
duplicated! 322
duplicated! 325
duplicated! 326
duplicated! 327
duplicated! 328
duplicated! 329
duplicated! 330
duplicated! 364
duplicated! 369
duplicated! 370
duplicated! 371
duplicated! 372
duplicated! 373
duplicated! 374
duplicated! 375
duplicated! 378
duplicated! 379
duplicated! 380
duplicated!

In [494]:
X_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,142,143,144,145,146,147,148,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,180,181,182,183,184,185,187,188,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,227,228,238,239,244,245,246,247,248,249,250,251,252,253,254,255,267,268,269,270,271,272,273,274,275,277,278,279,280,281,282,283,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,316,317,318,319,320,321,323,324,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,365,366,367,368,376,377,382,383,384,385,386,387,388,389,390,391,392,393,405,406,407,408,409,410,411,412,413,415,416,417,418,419,420,421,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,452,453,454,455,456,457,459,460,467,468,469,470,471,472,473,474,475,476,477,478,479,480,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,499,500,510,511,516,517,518,519,520,521,522,523,524,525,526,527,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589
425,3079.17,2405.56,2217.3777,1425.1041,1.7585,100.0,106.2556,0.1200,1.4794,-0.0198,-0.0004,0.9535,197.4874,0.0,8.0655,419.1988,10.2912,0.9719,187.1962,12.6006,1.3977,-6047.00,2653.50,-3526.00,-3200.6667,1.2963,2.0313,7.4395,70.0556,1.6778,0.1260,3.4416,85.2296,8.4421,50.2731,64.0899,49.7269,66.2998,86.8508,118.4058,79.06,2.035,70.0,365.0318,9.8550,142.3436,752.3565,1.2627,141.4400,1.0,648.8155,176.2604,4.647,4.902,2852.0,0.9211,0.9479,4.4571,-1.4345,364.1709,10.3000,120.9091,19.1810,21.4345,30.0664,722.2901,1.1383,144.5545,629.6345,105.8896,162.9708,466.6637,0.0,-0.0107,-0.0687,-0.0090,-0.0909,0.0262,-0.0881,-0.0239,-0.0079,7.0048,NaN,NaN,2.4439,0.9873,1780.6717,0.2009,8925.6302,-0.1635,0.0045,0.0026,0.0000,0.0001,-0.0523,0.4879,0.0465,-0.0006,0.0002,0.1649,-0.0127,-0.0006,-0.0045,-0.0026,0.1009,0.0222,NaN,NaN,NaN,0.4673,0.9319,0.0,733.3834,0.9907,58.1458,0.5985,0.9688,6.2889,15.87,2.854,15.87,15.88,1.3430,2.768,0.4947,3.265,-0.9461,0.8244,0.9995,2.3573,1008.0704,37.6658,129.0,57.5,147.0,61.1001,291.2710,0.3336,6.47,0.0034,0.0517,0.0648,0.0562,0.0187,5.8990,6.780,13.675,0.6360,0.0097,5.7757,0.62,0.0569,0.0587,2214.2998,2525.0,674.0,594.0,18577.0,0.164,0.075,0.167,2.3,1.2,0.195,0.478,1.0629,0.0748,0.4926,0.8627,0.4926,0.6733,0.3251,0.429,17.95,0.572,12.10,37.9610,0.0973,5.97,26.25,25.220,0.256,5.08,24.60,0.663,10.48,26.86,5.08,8.179,34.259,0.0970,7.66,0.0,26.86,113.467,0.0,0.0537,0.0627,0.0256,0.0625,0.0762,0.1026,0.1140,0.0518,3.3857,NaN,NaN,0.0486,0.0011,48.6101,0.0417,2240.2002,0.0198,0.0207,0.0040,0.0040,NaN,NaN,NaN,0.0263,0.0359,0.0,217.9194,0.0006,2.3805,0.0355,0.0193,0.4493,0.0935,17.406,2.9982,31.8096,20.1607,49.7806,18.5128,93.7489,0.1162,1.9384,0.0008,0.0169,0.0199,0.0177,0.0063,2.3444,1.8495,4.7039,0.1827,0.0032,2.2339,0.1013,0.0198,0.0151,693.3553,1064.3436,290.6424,276.5779,9336.8408,0.0676,0.0344,0.0776,0.6483,0.4147,0.0585,0.1410,0.4183,0.0303,0.1995,0.3072,0.1995,0.2606,0.1319,0.1565,5.2483,0.1720,3.3290,11.2091,0.0345,2.0798,7.9675,7.2798,0.0778,1.7799,7.4183,0.2211,3.2864,11.6297,1.7799,2.3034,11.3806,0.0296,2.4369,0.0,8.6853,31.3621,7.4494,2.7346,0.0,0.0142,0.0266,0.0114,0.0293,0.0226,0.0545,0.0549,0.0261,0.9775,NaN,NaN,0.0156,0.0003,18.

### Create feature Handler

In [495]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier


def NullDropHandler(train, test):
    tmp_stack = []
    for column in train.columns:
        if train[column].isnull().sum() > 500:
            train = train.drop([column], axis=1)
            test = test.drop([column], axis=1)
            tmp_stack.append(column)
    return train, test


def NaNValueHandler(df: pd.DataFrame):
    df = df.ffill()
    df = df.bfill()
    return df


def drop_zero_columns(train, test):
    zero_columns = train.columns[(train.mean() == 0) & (train.max() == 0) & (train.min() == 0)]
    train = train.drop(zero_columns, axis=1)
    test = test.drop(zero_columns, axis=1)
    return train, test


def augment_data_with_smote(X, y):
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X, y)

    return X_resampled, y_resampled


'''def LogTransform(train, test):
    numeric_features = train.select_dtypes(include=['int64', 'float64']).columns
    # Use np.where to handle negative values before applying log
    train[numeric_features] = train[numeric_features].apply(lambda x: np.log1p(np.where(x < 0, 0, x)))
    test[numeric_features] = test[numeric_features].apply(lambda x: np.log1p(np.where(x < 0, 0, x)))
    return train, test'''


def DataHandler(X_train, X_test, y_train, y_test):
    X_train, X_test = NullDropHandler(X_train.copy(), X_test.copy())
    X_train = NaNValueHandler(X_train)
    X_test = NaNValueHandler(X_test)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    X_train, X_test = drop_zero_columns(pd.DataFrame(X_train), pd.DataFrame(X_test))
    X_train, y_train = augment_data_with_smote(X_train, y_train)

    return X_train, X_test, np.array(y_train).ravel(), np.array(y_test).ravel()


# Usage
X_train, X_test, y_train, y_test = DataHandler(X_train, X_test, y_train, y_test)

X_train, X_test, y_train, y_test = pd.DataFrame(X_train), pd.DataFrame(X_test), pd.DataFrame(y_train), pd.DataFrame(
    y_test)

In [496]:
print('is NaN data check : ', X_train.isna().sum().sum())
print('is NaN data check : ', X_test.isna().sum().sum())

is NaN data check :  0
is NaN data check :  0


In [497]:
X_train

,0,1,2,3,4,6,7,8,9,10,11,12,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,43,44,45,46,47,48,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445
0,0.881309,-1.100675,0.568447,0.040984,-0.049872,0.814089,-0.191043,0.209377,-1.238595,-0.079477,-1.136553,-0.937733,-0.324219,0.319827,0.896176,0.049547,-1.090066,0.557840,-0.428272,-0.707400,-0.182523,0.208060,-0.991271,0.520545,0.512215,0.645343,0.162350,-1.661295,-1.751738,-0.440579,-0.057367,-0.395782,-0.260396,-0.181710,0.260388,0.248598,0.026608,-0.160686,0.456429,-0.644677,1.493125,-1.002310,0.692259,1.513724,0.460261,0.351222,1.905556,0.318467,0.914723,1.006248,-0.162706,-1.092913,-0.320540,-1.581288,-0.444881,1.473441,-0.452334,0.485562,0.674980,0.161596,0.407968,1.331497,-0.050526,-0.675341,1.083767,0.047208,-0.028262,-0.189134,-1.185784,-0.051758,-1.602507,0.989251,-1.432777,-0.161369,-0.398769,-0.871991,-1.122755,1.123041,0.394008,-0.518041,0.292253,0.237443,-1.896192,1.235162,1.032634,0.167198,0.368482,-0.314719,1.186038,0.667120,-1.592235,0.947758,2.565863,-0.947635,-0.635917,-1.245067,-1.041669,1.160960,0.396620,-1.126109,-0.066202,-0.286909,0.374717,-0.072608,0.000573,-0.205567,-0.165382,0.734198,-1.153699,0.369930,0.742847,0.569083,0.048914,-1.131818,0.289233,-0.342543,0.959861,0.747329,0.736584,0.630287,-0.571539,0.179768,-1.511634,0.473979,0.284541,-0.473322,-0.056581,-0.052900,-0.579828,-1.123770,0.025767,0.046630,0.128799,-0.853836,0.009485,-0.023705,0.028232,-0.235304,-0.749199,0.107282,-0.023097,1.678959,0.202213,-0.802854,2.084453,0.211651,-0.213811,-0.204129,-0.460162,-0.056761,1.486234,0.403142,2.438356,-0.737416,2.466368,2.988323,2.466254,-0.901340,1.082049,0.118994,-0.313755,0.130364,0.321566,1.677895,-0.557452,-0.415463,1.885922,-0.815157,-0.070801,-0.219214,0.385788,0.197066,-0.070796,0.982281,-0.539487,-0.133406,0.218293,-0.056765,-0.122101,-0.028262,0.334107,1.481447,-0.028262,-0.825771,0.229448,-0.803080,0.052085,-0.123581,0.688870,1.173203,-0.472138,-0.338448,-0.470280,-0.526006,-0.146485,-1.271712,-0.247341,2.726468,0.056176,0.243575,-0.449479,-0.397741,0.298558,-0.057496,1.873927,-0.101330,-0.037087,0.159311,0.594094,0.378485,0.784339,-0.293162,-0.689044,0.287935,-1.472698,0.486610,-0.417518,-0.496786,-0.056563,-0.222449,-0.906651,-1.204307,0.273077,0.465436,0.157320,-0.422983,-0.264769,0.029734,-0.014730,-0.243556,-0.420201,-0.080516,-0.014514,1.405691,0.129601,-0.796170,2.141075,0.071772,-0.192581,-0.181105,-0.816608,0.056617,1.256470,0.142782,2.702491,-0.7

In [498]:
tmp = pd.DataFrame(y_train)
tmp.value_counts()

0 
-1    1170
 1    1170
Name: count, dtype: int64

In [499]:
print(X_train.columns[(X_train.mean() == 0) & (X_train.max() == 0) & (X_train.min() == 0)])

Index([], dtype='int64')


In [500]:
X_train

,0,1,2,3,4,6,7,8,9,10,11,12,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,43,44,45,46,47,48,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445
0,0.881309,-1.100675,0.568447,0.040984,-0.049872,0.814089,-0.191043,0.209377,-1.238595,-0.079477,-1.136553,-0.937733,-0.324219,0.319827,0.896176,0.049547,-1.090066,0.557840,-0.428272,-0.707400,-0.182523,0.208060,-0.991271,0.520545,0.512215,0.645343,0.162350,-1.661295,-1.751738,-0.440579,-0.057367,-0.395782,-0.260396,-0.181710,0.260388,0.248598,0.026608,-0.160686,0.456429,-0.644677,1.493125,-1.002310,0.692259,1.513724,0.460261,0.351222,1.905556,0.318467,0.914723,1.006248,-0.162706,-1.092913,-0.320540,-1.581288,-0.444881,1.473441,-0.452334,0.485562,0.674980,0.161596,0.407968,1.331497,-0.050526,-0.675341,1.083767,0.047208,-0.028262,-0.189134,-1.185784,-0.051758,-1.602507,0.989251,-1.432777,-0.161369,-0.398769,-0.871991,-1.122755,1.123041,0.394008,-0.518041,0.292253,0.237443,-1.896192,1.235162,1.032634,0.167198,0.368482,-0.314719,1.186038,0.667120,-1.592235,0.947758,2.565863,-0.947635,-0.635917,-1.245067,-1.041669,1.160960,0.396620,-1.126109,-0.066202,-0.286909,0.374717,-0.072608,0.000573,-0.205567,-0.165382,0.734198,-1.153699,0.369930,0.742847,0.569083,0.048914,-1.131818,0.289233,-0.342543,0.959861,0.747329,0.736584,0.630287,-0.571539,0.179768,-1.511634,0.473979,0.284541,-0.473322,-0.056581,-0.052900,-0.579828,-1.123770,0.025767,0.046630,0.128799,-0.853836,0.009485,-0.023705,0.028232,-0.235304,-0.749199,0.107282,-0.023097,1.678959,0.202213,-0.802854,2.084453,0.211651,-0.213811,-0.204129,-0.460162,-0.056761,1.486234,0.403142,2.438356,-0.737416,2.466368,2.988323,2.466254,-0.901340,1.082049,0.118994,-0.313755,0.130364,0.321566,1.677895,-0.557452,-0.415463,1.885922,-0.815157,-0.070801,-0.219214,0.385788,0.197066,-0.070796,0.982281,-0.539487,-0.133406,0.218293,-0.056765,-0.122101,-0.028262,0.334107,1.481447,-0.028262,-0.825771,0.229448,-0.803080,0.052085,-0.123581,0.688870,1.173203,-0.472138,-0.338448,-0.470280,-0.526006,-0.146485,-1.271712,-0.247341,2.726468,0.056176,0.243575,-0.449479,-0.397741,0.298558,-0.057496,1.873927,-0.101330,-0.037087,0.159311,0.594094,0.378485,0.784339,-0.293162,-0.689044,0.287935,-1.472698,0.486610,-0.417518,-0.496786,-0.056563,-0.222449,-0.906651,-1.204307,0.273077,0.465436,0.157320,-0.422983,-0.264769,0.029734,-0.014730,-0.243556,-0.420201,-0.080516,-0.014514,1.405691,0.129601,-0.796170,2.141075,0.071772,-0.192581,-0.181105,-0.816608,0.056617,1.256470,0.142782,2.702491,-0.7

In [501]:
X_train.shape

(2340, 442)

In [502]:
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

In [503]:
from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(random_state=1234)
rfe = RFE(lr, n_features_to_select=300)
X_train_rfe = rfe.fit_transform(X_train, y_train)
X_test_rfe = rfe.transform(X_test)

pca = PCA(n_components=150)
X_train = pca.fit_transform(X_train_rfe)
X_test = pca.transform(X_test_rfe)

In [504]:
y_train = np.where(y_train == -1, 0, 1)
y_test = np.where(y_test == -1, 0, 1)

In [505]:
tmp = pd.DataFrame(y_train)
tmp.value_counts()

0
0    1170
1    1170
Name: count, dtype: int64

In [506]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Convert the classes parameter to a numpy array

y_train = pd.DataFrame(y_train)

model = xgb.XGBClassifier(
    max_depth=5,
    learning_rate=0.05,
    min_child_weight=1,
    n_estimators=100,
    subsample=0.8,
    colsample_bytree=0.8,

)

model.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [507]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X_train, y_train, cv=5, scoring='f1')
print(f'Cross-validated F1 scores: {scores}')

Cross-validated F1 scores: [0.98268398 0.9978678  1.         0.99148936 0.99574468]


In [508]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import balanced_accuracy_score

param_grid = {
    'max_depth': [4, 5, 6, 7],
    'learning_rate': [0.1, 0.05],
    'n_estimators': [50, 100, 150],
    'min_child_weight': [1, 2, 3]
}

In [510]:
gs = GridSearchCV(model, param_grid, scoring='balanced_accuracy', cv=2, verbose=2)
gs.fit(X_train, y_train)
gs.best_params_

Fitting 2 folds for each of 72 candidates, totalling 144 fits
[CV] END learning_rate=0.1, max_depth=4, min_child_weight=1, n_estimators=50; total time=   0.3s
[CV] END learning_rate=0.1, max_depth=4, min_child_weight=1, n_estimators=50; total time=   0.1s
[CV] END learning_rate=0.1, max_depth=4, min_child_weight=1, n_estimators=100; total time=   0.1s
[CV] END learning_rate=0.1, max_depth=4, min_child_weight=1, n_estimators=100; total time=   0.1s
[CV] END learning_rate=0.1, max_depth=4, min_child_weight=1, n_estimators=150; total time=   0.1s
[CV] END learning_rate=0.1, max_depth=4, min_child_weight=1, n_estimators=150; total time=   2.0s
[CV] END learning_rate=0.1, max_depth=4, min_child_weight=2, n_estimators=50; total time=   0.1s
[CV] END learning_rate=0.1, max_depth=4, min_child_weight=2, n_estimators=50; total time=   0.1s
[CV] END learning_rate=0.1, max_depth=4, min_child_weight=2, n_estimators=100; total time=   0.1s
[CV] END learning_rate=0.1, max_depth=4, min_child_weight=2,

{'learning_rate': 0.1,
 'max_depth': 4,
 'min_child_weight': 1,
 'n_estimators': 150}

In [564]:
model = xgb.XGBClassifier(
    max_depth=8,
    learning_rate=0.05,
    min_child_weight=2,
    n_estimators=70,
    subsample=0.8,
    colsample_bytree=0.8,

)
model.fit(X_train,y_train)
y_pred_proba = model.predict_proba(X_test)[:, 1]

threshold = 0.25  
y_pred = (y_pred_proba >= threshold).astype(int)

In [565]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.93      0.98      0.96       293
           1       0.14      0.05      0.07        21

    accuracy                           0.92       314
   macro avg       0.54      0.51      0.51       314
weighted avg       0.88      0.92      0.90       314


In [ ]:
import joblib

joblib.dump(model, 'best_xgboost_model.pkl')